In [1]:
library(tidyverse)
library(brms)

── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()

Loading required package: Rcpp

Loading 'brms' package (version 2.14.4). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:


In [2]:
data_cues <- read_csv("../data/data_cues.txt")



── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  Glottocode = col_character(),
  Language = col_character(),
  Genus = col_character(),
  Case_Marking = col_double(),
  Tight_Semantics = col_double(),
  Rigid_Order = col_double(),
  Verb_Middle = col_double()
)



In [3]:
trees <- ape::read.tree("../data/posterior.tree")
tree <- phangorn::maxCladeCred(trees)

In [4]:

tree_cor <- ape::vcv.phylo(tree, corr = TRUE)
tree_cor <- tree_cor[data_cues$Language, data_cues$Language]


In [5]:
## first we just do scaling

data_cues <- data_cues %>%
    mutate(
        x1 = scale(Case_Marking)
      , x2 = scale(Tight_Semantics)
      , x3 = scale(Rigid_Order)
      , x4 = scale(Verb_Middle))

In [6]:
## the baseline correlations

round(cor(data_cues[,8:11]), 2)

,x1,x2,x3,x4
x1,1.00,0.35,-0.77,-0.19
x2,0.35,1.00,-0.18,-0.37
x3,-0.77,-0.18,1.00,0.18
x4,-0.19,-0.37,0.18,1.00


In [7]:

rescale <- function(x, e = 0.0001) {
    y = ifelse(x == 1, x - e, x)
    ifelse(y == 0, y + e, y)
}


In [8]:
data_cues_3 <- data_cues %>%
    mutate(
        x1 = rescale(Case_Marking)
      , x2 = rescale(Tight_Semantics)
      , x3 = rescale(Rigid_Order)
      , x4 = rescale(Verb_Middle))


In [23]:
bm_2b <- brm(bf(mvbind(x1,x2,x3,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_2b

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling



 Family: MV(beta, beta, beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity
         mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.53      0.37     0.88     2.33 1.00
sd(x2_Intercept)                   0.25      0.14     0.02     0.56 1.00
sd(x3_Intercept)                   1.55      0.26     1.02     2.06 1.00
sd(x4_Intercept)                   2.80      0.41     2.00     3.62 1.00
cor(x1_Intercep

In [11]:
bm_3_1 <- brm(bf(mvbind(x1,x2) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_1

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 5 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 5 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.30      0.39     0.55     2.14 1.00
sd(x2_Intercept)                   0.29      0.15     0.03     0.59 1.00
cor(x1_Intercept,x2_Intercept)     0.45      0.35    -0.37     0.96 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   2183     3263
sd(x2_Intercept)                   1402     2924
cor(x1_Intercept,x2_Intercept)     4516     3593

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [12]:
bm_3_2 <- brm(bf(mvbind(x1,x3) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_2

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 8 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 8 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.43      0.35     0.83     2.19 1.00
sd(x3_Intercept)                   1.54      0.27     0.99     2.08 1.00
cor(x1_Intercept,x3_Intercept)    -0.83      0.13    -0.99    -0.51 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   3437     5168
sd(x3_Intercept)                   1533     2836
cor(x1_Intercept,x3_Intercept)     2478     4987

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [13]:
bm_3_3 <- brm(bf(mvbind(x1,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_3

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 6 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 6 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.34      0.41     0.57     2.19 1.00
sd(x4_Intercept)                   2.74      0.45     1.77     3.56 1.01
cor(x1_Intercept,x4_Intercept)    -0.44      0.20    -0.78    -0.02 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   1914     2371
sd(x4_Intercept)                   1084      856
cor(x1_Intercept,x4_Intercept)     1715     2297

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [14]:
bm_3_4 <- brm(bf(mvbind(x2,x3) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_4

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 229 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#bulk-ess”
Warning message:
“Tail Effective Samples Size (ESS) is too low, 

 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x2_Intercept)                   0.25      0.16     0.01     0.58 1.00
sd(x3_Intercept)                   1.54      0.32     0.86     2.12 1.01
cor(x2_Intercept,x3_Intercept)    -0.25      0.39    -0.93     0.60 1.01
                               Bulk_ESS Tail_ESS
sd(x2_Intercept)                    956     3320
sd(x3_Intercept)                    779     1401
cor(x2_Intercept,x3_Intercept)      792     1093

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [15]:
bm_3_5 <- brm(bf(mvbind(x2,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x2) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_5

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 42 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 42 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x2 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x2_Intercept)                   0.29      0.15     0.03     0.57 1.00
sd(x4_Intercept)                   2.65      0.55     1.20     3.58 1.00
cor(x2_Intercept,x4_Intercept)    -0.43      0.29    -0.94     0.19 1.00
                               Bulk_ESS Tail_ESS
sd(x2_Intercept)                   1250     2175
sd(x4_Intercept)                    760      530
cor(x2_Intercept,x4_Intercept)     1023     1359

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [16]:
bm_3_6 <- brm(bf(mvbind(x3,x4) ~ 1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_3_6

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 10 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“There were 2 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 10 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x3 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ 1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x3_Intercept)                   1.59      0.28     1.03     2.15 1.00
sd(x4_Intercept)                   2.77      0.45     1.81     3.61 1.01
cor(x3_Intercept,x4_Intercept)     0.38      0.16     0.05     0.65 1.00
                               Bulk_ESS Tail_ESS
sd(x3_Intercept)                   1210     1872
sd(x4_Intercept)                   1260     1168
cor(x3_Intercept,x4_Intercept)     3268     4944

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat Bul

In [18]:
bm_4_1 <- brm(bf(mvbind(x1,x3) ~ x4
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo                     
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_4_1

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 25 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 25 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ x4 + (1 | p | gr(Language, cov = phylo)) 
         x3 ~ x4 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   1.43      0.36     0.80     2.22 1.00
sd(x3_Intercept)                   1.49      0.25     1.00     2.00 1.00
cor(x1_Intercept,x3_Intercept)    -0.77      0.15    -0.98    -0.42 1.00
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                   2635     4447
sd(x3_Intercept)                   1799     3107
cor(x1_Intercept,x3_Intercept)     2139     3910

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat B

In [19]:
bm_4_2 <- brm(bf(mvbind(x1,x4) ~ x3
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x1) # priors phylo                     
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_4_2

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 30 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 30 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x1 ~ x3 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ x3 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x1_Intercept)                   0.79      0.50     0.04     1.79 1.01
sd(x4_Intercept)                   2.69      0.38     1.96     3.48 1.00
cor(x1_Intercept,x4_Intercept)    -0.31      0.34    -0.91     0.56 1.01
                               Bulk_ESS Tail_ESS
sd(x1_Intercept)                    666     2178
sd(x4_Intercept)                   1958     2478
cor(x1_Intercept,x4_Intercept)      548      574

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat B

In [20]:
bm_4_3 <- brm(bf(mvbind(x3,x4) ~ x1
              + (1|p|gr(Language, cov = phylo))) # phylo term with correlations
         , prior = c(prior(normal(0, 1), class = Intercept) # prior Intercept
                   , prior(normal(0, 1), class = sd, resp = x3) # priors phylo                     
                   , prior(normal(0, 1), class = sd, resp = x4) # priors phylo
                     )
         , family = Beta
         , data = data_cues_3
         , data2 = list(phylo = tree_cor)
         , cores = 4, chains = 4
         , iter = 4000 # this model needs a lot more iters
         , warmup = 1000, seed = 1234
         , control = list(max_treedepth = 10
                        , adapt_delta = 0.95))

bm_4_3

Setting 'rescor' to FALSE by default for this model

Warning message:
“Specifying global priors for regression coefficients in multivariate models is deprecated. Please specify priors separately for each response variable.”
Compiling Stan program...

Start sampling

Warning message:
“There were 3 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them.”
Warning message:
“Examine the pairs() plot to diagnose sampling problems
”
Warning message:
“There were 3 divergent transitions after warmup. Increasing adapt_delta above 0.95 may help. See http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup”


 Family: MV(beta, beta) 
  Links: mu = logit; phi = identity
         mu = logit; phi = identity 
Formula: x3 ~ x1 + (1 | p | gr(Language, cov = phylo)) 
         x4 ~ x1 + (1 | p | gr(Language, cov = phylo)) 
   Data: data_cues_3 (Number of observations: 30) 
Samples: 4 chains, each with iter = 4000; warmup = 1000; thin = 1;
         total post-warmup samples = 12000

Group-Level Effects: 
~Language (Number of levels: 30) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(x3_Intercept)                   1.18      0.27     0.58     1.68 1.01
sd(x4_Intercept)                   2.69      0.39     1.95     3.46 1.00
cor(x3_Intercept,x4_Intercept)     0.20      0.19    -0.18     0.53 1.00
                               Bulk_ESS Tail_ESS
sd(x3_Intercept)                   1118     1272
sd(x4_Intercept)                   1800     2404
cor(x3_Intercept,x4_Intercept)     3416     4505

Population-Level Effects: 
             Estimate Est.Error l-95% CI u-95% CI Rhat B